In [1]:
import pandas as pd 
import datetime
data = pd.read_csv('data.csv')
user_id_list = data['user_id'].unique()
user_id_list

array([    0,     1,     2, ..., 25068, 25069, 25070])

In [2]:
new_data = {'user_id': user_id_list, 'car_ownership': 0}
df = pd.DataFrame(new_data)
df

,user_id,car_ownership
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
25066,25066,0
25067,25067,0
25068,25068,0
25069,25069,0


In [3]:
tripleg = pd.read_csv('triplegs.csv')
stop_points = pd.read_csv('staypoints.csv')

In [4]:
stop_points['started_at'] = pd.to_datetime(stop_points['started_at'])
tripleg['finished_at'] = pd.to_datetime(tripleg['finished_at'])

In [5]:
ID_list=stop_points['user_id'].unique()
len(ID_list)

10987

In [6]:
time_window = datetime.timedelta(minutes=5)

miss = 0
check_len = 0
data_to_append_list = []
for id in ID_list:
    related_stop_by_user = stop_points[stop_points['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for stop_related_row in related_stop_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > stop_related_row[3] - time_window) &
            (related_tripled_by_user['finished_at'] <= stop_related_row[3])
        ]
        #print(len(tripled_found))
        if len(tripled_found) == 0: continue #did not found related trip
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        data_to_append = {'user_id': id, 'trip_started_at': tripled_found.iloc[0]['started_at'],
                'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                'trip':tripled_found.iloc[0]['geom']}
        
        data_to_append_list.append(data_to_append)
        

trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'trip_started_at', 'trip_finished_at', 'trip'])
trip_df

,user_id,trip_started_at,trip_finished_at,trip
0,4,2022-09-23 12:15:13-04:00,2022-09-23 12:15:20-04:00,LINESTRING (-81.7982300000000038 30.0503700000...
1,5,2022-10-07 22:00:05-04:00,2022-10-07 22:00:34-04:00,LINESTRING (-81.6504979999999989 30.4700999999...
2,5,2022-10-08 13:45:17-04:00,2022-10-08 13:45:27-04:00,LINESTRING (-81.6341471700000056 30.4783272700...
3,5,2022-10-09 13:00:02-04:00,2022-10-09 13:00:51-04:00,LINESTRING (-81.6210899999999953 30.4746300000...
4,5,2022-10-11 22:00:32-04:00,2022-10-11 22:00:46-04:00,LINESTRING (-81.6504999999999939 30.4700999999...
...,...,...,...,...
49291,25048,2022-10-30 01:52:42-04:00,2022-10-30 01:54:53-04:00,LINESTRING (-82.7853083600000019 30.2982544900...
49292,25048,2022-10-30 02:08:36-04:00,2022-10-30 02:08:46-04:00,LINESTRING (-82.9850299999999947 30.4801399999...
49293,25048,2022-10-30 02:08:36-04:00,2022-10-30 02:08:46-04:00,LINESTRING (-82.9850299999999947 30.4801399999...
49294,25049,2022-10-26 23:03:49-04:00,2022-10-26 23:04:01-04:00,LINESTRING (-81.5443425800000057 30.3223002299...


In [7]:
trip_df.to_csv('stop_related_trip.csv')

In [8]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas
import trackintel as ti
import numpy as np
tpls = ti.io.file.read_triplegs_csv('stop_related_trip.csv', columns={'trip_started_at':'started_at', 'trip_finished_at':'finished_at', 'trip':'geom'}, crs='EPSG:4326', index_col=0)

In [9]:
# Predict the transport mode of triplegs.
"""simple-coarse method includes {'slow_mobility', 'motorized_mobility', 'fast_mobility'}. 
In the default classification, slow_mobility (<15 km/h) includes transport modes such as walking or cycling, 
motorized_mobility (<100 km/h) modes such as car or train, 
and fast_mobility (>100 km/h) modes such as high-speed rail or airplanes."""
categories = {
    15/3.6: 'slow_mobility',  # walking or cycling
    193/3.6: 'motorized_mobility',  # car
    np.inf: 'fast_mobility'  # high-speed rail or airplane
}
mode_tpls  = tpls.as_triplegs.predict_transport_mode(categories = categories)
mode_tpls

/usr/local/lib/python3.9/site-packages/pygeos/io.py:89: UserWarning: The shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the PyGEOS GEOS version (3.10.4-CAPI-1.16.2). Conversions between both will be slow
  warnings.warn(


,user_id,started_at,finished_at,geom,mode
0,4,2022-09-23 12:15:13-04:00,2022-09-23 12:15:20-04:00,"LINESTRING (-81.79823 30.05037, -81.79821 30.0...",slow_mobility
1,5,2022-10-07 22:00:05-04:00,2022-10-07 22:00:34-04:00,"LINESTRING (-81.65050 30.47010, -81.63820 30.4...",motorized_mobility
2,5,2022-10-08 13:45:17-04:00,2022-10-08 13:45:27-04:00,"LINESTRING (-81.63415 30.47833, -81.63416 30.4...",slow_mobility
3,5,2022-10-09 13:00:02-04:00,2022-10-09 13:00:51-04:00,"LINESTRING (-81.62109 30.47463, -81.62109 30.4...",slow_mobility
4,5,2022-10-11 22:00:32-04:00,2022-10-11 22:00:46-04:00,"LINESTRING (-81.65050 30.47010, -81.65050 30.4...",slow_mobility
...,...,...,...,...,...
49291,25048,2022-10-30 01:52:42-04:00,2022-10-30 01:54:53-04:00,"LINESTRING (-82.78531 30.29825, -82.81831 30.3...",motorized_mobility
49292,25048,2022-10-30 02:08:36-04:00,2022-10-30 02:08:46-04:00,"LINESTRING (-82.98503 30.48014, -82.98502 30.4...",slow_mobility
49293,25048,2022-10-30 02:08:36-04:00,2022-10-30 02:08:46-04:00,"LINESTRING (-82.98503 30.48014, -82.98502 30.4...",slow_mobility
49294,25049,2022-10-26 23:03:49-04:00,2022-10-26 23:04:01-04:00,"LINESTRING (-81.54434 30.32230, -81.55156 30.3...",fast_mobility


In [10]:
mode_split = ti.analysis.modal_split.calculate_modal_split(tpls=mode_tpls, metric='count', per_user=True)
mode_split

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
4,0,0,1
5,0,1,3
8,0,1,0
14,3,73,30
20,0,2,7
...,...,...,...
24999,0,3,0
25028,0,0,1
25048,0,7,7


In [11]:
index = mode_split.index.values
for id in index:
    motor_count = mode_split.loc[id, 'fast_mobility'] + mode_split.loc[id, 'motorized_mobility'] + mode_split.loc[id, 'slow_mobility']
    if mode_split.loc[id, 'motorized_mobility'] / motor_count > 0.5:
        df.loc[df['user_id'] == id, 'car_ownership'] = 1
df

,user_id,car_ownership
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
25066,25066,0
25067,25067,0
25068,25068,0
25069,25069,0


In [12]:
print(len(df[df['car_ownership'] == 1]))

2345


In [13]:
home_location = pd.read_csv('home_location.csv', index_col=0)
home_location

,ID,LAT,LON,LAT-4326,LON-4326
0,0,3401080.0,-9141180.0,29.199637,-82.116617
1,1,3458860.0,-9093020.0,29.651722,-81.683988
2,2,3456360.0,-9163720.0,29.632203,-82.319097
3,3,3531380.0,-9073260.0,30.216272,-81.506481
4,4,3510020.0,-9105740.0,30.050323,-81.798254
...,...,...,...,...,...
20950,25043,3372900.0,-9048460.0,28.978423,-81.283699
20951,25052,3584140.0,-9066040.0,30.624974,-81.441623
20952,25054,3490020.0,-9051880.0,29.894687,-81.314422
20953,25057,3545180.0,-9097480.0,30.323338,-81.724053


In [14]:
from geopy.distance import geodesic
from shapely.wkt import loads
from tqdm import tqdm
data_to_append_list = []
home_id_list = home_location['ID'].unique()
for id in tqdm(home_id_list, total = len(home_id_list)):
    # print(home_location.loc[home_location['ID'] == id, 'LAT-4326'].values[0])
    p1 = (home_location.loc[home_location['ID'] == id, "LAT-4326"].values[0], home_location.loc[home_location['ID'] == id, "LON-4326"].values[0])
    # print(p1)
    for sp in stop_points[stop_points['user_id'] == id].itertuples():
        p = loads(sp[5])
        lat = p.y
        lon = p.x
        p2 = (lat, lon)
        # print(p2)
        if geodesic(p1, p2).meters < 100:
            data_to_append_list.append(sp)
new_staypoints = pd.DataFrame.from_records(data_to_append_list)
new_staypoints


100%|██████████| 20955/20955 [01:41<00:00, 207.03it/s]


,0,1,2,3,4,5
0,1,1,1,2022-09-06 22:19:08-04:00,2022-09-06 22:22:07-04:00,POINT (-81.6840639999999922 29.6517550000000014)
1,3,3,1,2022-09-06 22:31:37-04:00,2022-09-06 22:34:49-04:00,POINT (-81.6840650000000181 29.6517550000000014)
2,16,16,2,2022-09-05 19:40:35-04:00,2022-09-05 20:01:05-04:00,POINT (-82.3190549210000029 29.6321820860000074)
3,18,18,2,2022-09-07 19:39:17-04:00,2022-09-07 20:02:10-04:00,POINT (-82.3190553007142825 29.6321777757142861)
4,20,20,2,2022-09-08 19:40:41-04:00,2022-09-08 20:01:10-04:00,POINT (-82.3190579438461469 29.6321850161538478)
...,...,...,...,...,...,...
62079,396535,396535,24506,2022-10-24 10:50:50-04:00,2022-10-24 10:57:41-04:00,POINT (-82.3449399999999940 29.5589399999999998)
62080,396540,396540,24506,2022-10-26 10:50:02-04:00,2022-10-26 10:54:05-04:00,POINT (-82.3449470049999945 29.5589244700000009)
62081,396547,396547,24506,2022-10-29 10:50:38-04:00,2022-10-29 10:59:48-04:00,POINT (-82.3449399999999940 29.5589300000000001)
62082,397089,397089,24732,2022-10-24 12:32:31-04:00,2022-10-24 12:45:35-04:00,POINT (-81.6479450000000071 28.9665500000000016)


In [15]:
drop_cols = [0,1]
new_staypoint = new_staypoints.drop(new_staypoints.columns[drop_cols], axis=1)
new_staypoint = new_staypoint.rename(columns={2:'user_id', 3: 'started_at', 4: 'finished_at', 5: 'geom'})
new_staypoint


,user_id,started_at,finished_at,geom
0,1,2022-09-06 22:19:08-04:00,2022-09-06 22:22:07-04:00,POINT (-81.6840639999999922 29.6517550000000014)
1,1,2022-09-06 22:31:37-04:00,2022-09-06 22:34:49-04:00,POINT (-81.6840650000000181 29.6517550000000014)
2,2,2022-09-05 19:40:35-04:00,2022-09-05 20:01:05-04:00,POINT (-82.3190549210000029 29.6321820860000074)
3,2,2022-09-07 19:39:17-04:00,2022-09-07 20:02:10-04:00,POINT (-82.3190553007142825 29.6321777757142861)
4,2,2022-09-08 19:40:41-04:00,2022-09-08 20:01:10-04:00,POINT (-82.3190579438461469 29.6321850161538478)
...,...,...,...,...
62079,24506,2022-10-24 10:50:50-04:00,2022-10-24 10:57:41-04:00,POINT (-82.3449399999999940 29.5589399999999998)
62080,24506,2022-10-26 10:50:02-04:00,2022-10-26 10:54:05-04:00,POINT (-82.3449470049999945 29.5589244700000009)
62081,24506,2022-10-29 10:50:38-04:00,2022-10-29 10:59:48-04:00,POINT (-82.3449399999999940 29.5589300000000001)
62082,24732,2022-10-24 12:32:31-04:00,2022-10-24 12:45:35-04:00,POINT (-81.6479450000000071 28.9665500000000016)


In [16]:
time_window = datetime.timedelta(minutes=5)

miss = 0
check_len = 0
data_to_append_list = []
for id in ID_list:
    related_stop_by_user = new_staypoint[new_staypoint['user_id'] == id]
    related_tripled_by_user = tripleg[tripleg['user_id'] == id]
    if len(related_tripled_by_user) == 0: 
        miss += 1
        continue
    
    for stop_related_row in related_stop_by_user.itertuples():
        tripled_found = related_tripled_by_user[
            (related_tripled_by_user['finished_at'] > stop_related_row[2] - time_window) &
            (related_tripled_by_user['finished_at'] <= stop_related_row[2])
        ]
        #print(len(tripled_found))
        if len(tripled_found) == 0: continue #did not found related trip
        #check if there if repeat data
        if len(tripled_found) > 1: 
            check_len += 1
            tripled_found = tripled_found.nlargest(1, 'finished_at')
        
        data_to_append = {'user_id': id, 'trip_started_at': tripled_found.iloc[0]['started_at'],
                'trip_finished_at': tripled_found.iloc[0]['finished_at'],
                'trip':tripled_found.iloc[0]['geom']}
        
        data_to_append_list.append(data_to_append)
        

new_trip_df = pd.DataFrame(data_to_append_list, columns=['user_id', 'trip_started_at', 'trip_finished_at', 'trip'])
new_trip_df

,user_id,trip_started_at,trip_finished_at,trip
0,14,2022-09-09 20:33:27-04:00,2022-09-09 20:35:51-04:00,LINESTRING (-81.4925243999999935 30.3372013000...
1,14,2022-09-10 13:13:33-04:00,2022-09-10 13:13:44-04:00,LINESTRING (-81.4755399999999952 30.3195799999...
2,14,2022-09-10 13:49:34-04:00,2022-09-10 13:54:03-04:00,LINESTRING (-81.4898749999999978 30.3275840000...
3,14,2022-09-15 22:42:23-04:00,2022-09-15 22:43:24-04:00,LINESTRING (-81.4953000000000003 30.3301900000...
4,14,2022-09-26 15:47:07-04:00,2022-09-26 15:47:49-04:00,LINESTRING (-81.3932800000000043 30.3126800000...
...,...,...,...,...
4534,24509,2022-10-23 21:57:26-04:00,2022-10-23 21:58:03-04:00,LINESTRING (-81.7518132000000008 30.3788243999...
4535,24509,2022-10-25 01:20:59-04:00,2022-10-25 01:22:52-04:00,LINESTRING (-81.7429779999999937 30.3809520000...
4536,24509,2022-10-25 02:23:18-04:00,2022-10-25 02:25:12-04:00,LINESTRING (-81.7518210399999958 30.3788065899...
4537,24509,2022-10-25 03:19:35-04:00,2022-10-25 03:20:42-04:00,LINESTRING (-81.7520785300000057 30.3786778499...


In [17]:
new_trip_df.to_csv('home_related_trip.csv')

In [18]:
new_tpls = ti.io.file.read_triplegs_csv('home_related_trip.csv', columns={'trip_started_at':'started_at', 'trip_finished_at':'finished_at', 'trip':'geom'}, crs='EPSG:4326', index_col=0)
new_mode_tpls  = new_tpls.as_triplegs.predict_transport_mode(categories = categories)
new_mode_tpls

,user_id,started_at,finished_at,geom,mode
0,14,2022-09-09 20:33:27-04:00,2022-09-09 20:35:51-04:00,"LINESTRING (-81.49252 30.33720, -81.49291 30.3...",slow_mobility
1,14,2022-09-10 13:13:33-04:00,2022-09-10 13:13:44-04:00,"LINESTRING (-81.47554 30.31958, -81.47554 30.3...",slow_mobility
2,14,2022-09-10 13:49:34-04:00,2022-09-10 13:54:03-04:00,"LINESTRING (-81.48987 30.32758, -81.49529 30.3...",motorized_mobility
3,14,2022-09-15 22:42:23-04:00,2022-09-15 22:43:24-04:00,"LINESTRING (-81.49530 30.33019, -81.49530 30.3...",motorized_mobility
4,14,2022-09-26 15:47:07-04:00,2022-09-26 15:47:49-04:00,"LINESTRING (-81.39328 30.31268, -81.39326 30.3...",slow_mobility
...,...,...,...,...,...
4534,24509,2022-10-23 21:57:26-04:00,2022-10-23 21:58:03-04:00,"LINESTRING (-81.75181 30.37882, -81.75191 30.3...",motorized_mobility
4535,24509,2022-10-25 01:20:59-04:00,2022-10-25 01:22:52-04:00,"LINESTRING (-81.74298 30.38095, -81.74297 30.3...",motorized_mobility
4536,24509,2022-10-25 02:23:18-04:00,2022-10-25 02:25:12-04:00,"LINESTRING (-81.75182 30.37881, -81.75206 30.3...",slow_mobility
4537,24509,2022-10-25 03:19:35-04:00,2022-10-25 03:20:42-04:00,"LINESTRING (-81.75208 30.37868, -81.75208 30.3...",slow_mobility


In [19]:
new_mode_split = ti.analysis.modal_split.calculate_modal_split(tpls=new_mode_tpls, metric='count', per_user=True)
new_mode_split

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
14,0,6,5
51,0,2,2
76,0,5,0
83,0,0,1
87,0,0,1
...,...,...,...
24290,0,1,0
24308,0,1,0
24385,0,0,1


In [20]:
index = new_mode_split.index.values
for id in index:
    motor_count = new_mode_split.loc[id, 'motorized_mobility']
    if motor_count > 0:
        df.loc[df['user_id'] == id, 'car_ownership'] = 1
df

,user_id,car_ownership
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
25066,25066,0
25067,25067,0
25068,25068,0
25069,25069,0


In [21]:
print(len(df[df['car_ownership'] == 1]))

2669


In [26]:
all_tpls = ti.io.file.read_triplegs_csv('triplegs.csv', index_col=0)
all_mode_tpls  = all_tpls.as_triplegs.predict_transport_mode(categories = categories)
all_mode_tpls

/usr/local/lib/python3.9/site-packages/trackintel/geogr/distances.py:227: UserWarning: The CRS of your data is not defined.
  warnings.warn("The CRS of your data is not defined.")


,user_id,started_at,finished_at,geom,mode
id,,,,,
0,0,2022-09-06 13:11:03-04:00,2022-09-06 13:11:54-04:00,"LINESTRING (-82.07077 29.19301, -82.07077 29.1...",slow_mobility
4,0,2022-09-09 00:59:39-04:00,2022-09-09 00:59:49-04:00,"LINESTRING (-82.11661 29.19956, -82.11661 29.1...",slow_mobility
5,1,2022-09-06 09:15:08-04:00,2022-09-06 09:15:58-04:00,"LINESTRING (-81.67753 29.65504, -81.67751 29.6...",slow_mobility
6,1,2022-09-06 21:34:07-04:00,2022-09-06 21:34:57-04:00,"LINESTRING (-81.68406 29.65178, -81.68406 29.6...",slow_mobility
7,1,2022-09-07 00:05:48-04:00,2022-09-07 00:09:57-04:00,"LINESTRING (-81.65829 29.65349, -81.68407 29.6...",motorized_mobility
...,...,...,...,...,...
943571,25067,2022-10-27 01:06:08-04:00,2022-10-27 01:09:53-04:00,"LINESTRING (-81.61198 30.38249, -81.61198 30.3...",slow_mobility
943572,25068,2022-10-31 02:06:17-04:00,2022-10-31 02:07:50-04:00,"LINESTRING (-81.75724 30.33606, -81.75724 30.3...",slow_mobility
943573,25069,2022-10-26 08:25:34-04:00,2022-10-26 08:25:48-04:00,"LINESTRING (-82.09808 29.99144, -82.09809 29.9...",slow_mobility


In [27]:
all_mode_split = ti.analysis.modal_split.calculate_modal_split(tpls=all_mode_tpls, metric='count', per_user=True)
all_mode_split

mode,fast_mobility,motorized_mobility,slow_mobility
user_id,,,
0,0,0,2
1,0,9,23
2,0,0,126
3,0,0,1
4,1,5,418
...,...,...,...
25062,0,0,1
25065,0,0,6
25067,0,0,28


In [28]:
index = all_mode_split.index.values
for id in index:
    motor_count = all_mode_split.loc[id, 'motorized_mobility']
    if motor_count > 0:
        df.loc[df['user_id'] == id, 'car_ownership'] = 1
df

,user_id,car_ownership
0,0,0
1,1,1
2,2,0
3,3,0
4,4,1
...,...,...
25066,25066,0
25067,25067,0
25068,25068,0
25069,25069,0


In [29]:
home_locations = home_location.rename(columns={'ID':'user_id'})
home_locations
merged_data = pd.merge(df, home_locations[['user_id','LAT-4326','LON-4326']], on='user_id', how='left')
merged_data

,user_id,car_ownership,LAT-4326,LON-4326
0,0,0,29.199637,-82.116617
1,1,1,29.651722,-81.683988
2,2,0,29.632203,-82.319097
3,3,0,30.216272,-81.506481
4,4,1,30.050323,-81.798254
...,...,...,...,...
25066,25066,0,29.988720,-81.480610
25067,25067,0,30.379152,-81.605476
25068,25068,0,30.336054,-81.757291
25069,25069,0,29.409107,-82.247951


In [30]:
print(len(df[df['car_ownership'] == 1]))

6730


In [23]:
merged_data.to_csv('User.csv')